# Geocoding API


In [1]:
import herepy
import pandas as pd
from tqdm import tqdm 
import time
import math

## Tuguegarao

In [9]:
df = pd.read_csv('Mago_copy_CaVolun_Initiative_105720201119.csv')

In [10]:
df = df[(df['USABLE ADDRESS'] == 'Yes') | (df['REVISED ADDRESS'].notnull())]

In [11]:
df = df[df['RESCUE STATUS'] != 'Rescued']
df = df[df['RESCUE STATUS'] != 'Ongoing']
df = df[df['RESCUE STATUS'] != 'No need']
df = df[df['RESCUE STATUS'] != 'Safe at home']

In [12]:
df['RESCUE STATUS'].unique()

array([nan, 'Not yet responded', 'No info', 'Second floor na baha',
       'not yet rescued', 'NEEDS URGENT HELP', 'Wala nang battery',
       'trapped on the 2nd floor', '2 adults 1 kid, nasa bubong', '2:08',
       'Not Yet Responded', 'Cannot contact',
       'landmark: pinakadulo malapit sa evas',
       'light can be seen at jl fuel', 'URGENT! may inatake sa puso',
       'flashlight standby; di na kayang sumigaw',
       'no other info; not yet rescued since 8pm',
       'Families - sobrang gutom', 'cannot be rescued due to location',
       'need rescue asap'], dtype=object)

In [13]:
df = df[df['address'] != '']

In [14]:
df['AREA'] = df['AREA'].replace('Cetro', 'Centro')
df['AREA'] = df['AREA'].replace('North: Carig-Caritan', 'Carig-Caritan')
df['AREA'] = df['AREA'].replace('West: Buntun-Ugac', 'Buntun-Ugac')

In [15]:
df['AREA'].unique()

array(['Pengue', 'Centro', 'Carig-Caritan', nan, 'Linao Norte',
       'Balzain west', 'Centro: Centro 1-12', 'Buntun-Ugac',
       'Annafuan East', 'Annafunan East', 'Linao East', 'Linao east',
       'Liano EAST tapat', 'Brgy. San Gabriel', 'San Gabriel',
       'Linao North', 'Tuguegarao City', 'Enrile Cagayan', 'Buntun',
       'before meynars papasok sa drugstore', 'Tanza', 'Tan',
       'Liano East ', 'Balzain East', 'balzain east ',
       'Balzain East Tuguegarao City', 'Annafunan West', 'West Area',
       'Linao, Tuguegarao', 'Balzain', 'East: Namabbalan-Capatan',
       'Linao West', 'linao east ', 'Cattaggamman', 'Linao east ',
       'linao norte', 'Anao, Cabagan, Isabela',
       'Pallua Sur, near elem school papasok sa kanto ',
       'Linao Norte, Tuguegarao', 'LINAO NORTH', 'pinakadulo ',
       'Linao Norte ', 'Ugac Sur', 'Cataggaman Nuevo'], dtype=object)

In [16]:
df = df[df['address'] != '']

## Google Maps Geocoding

In [18]:
import googlemaps
from datetime import datetime

# PUT YOUR OWN API KEY
gmaps = googlemaps.Client(key='')

In [178]:
#Testing
geocode_result = gmaps.geocode('zone 4 duhat street barangay capatan ,Namabbalan-Capatan, Tuguegarao City, Cagayan')
geocode_result[0]

In [19]:
lat_list = []
lng_list = []
with tqdm(total=df.shape[0]) as pbar:
    for index, row in df.iterrows():
        pbar.update(1)
                
        try:
            if pd.isnull(row['REVISED ADDRESS']):
                address = row['address']
            else:
                address = row['REVISED ADDRESS']
        except:
            address = row['REVISED ADDRESS']
            
        if pd.isnull(row['AREA']) == False:
            address = address + ', ' + row['AREA']
            
        if "tuguegarao" in address.lower():
            address = address + ', Cagayan'
        else:
            address = address + ', Tuguegarao City, Cagayan'
            
#         print(address)
                    
        geocode_result = gmaps.geocode(address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
            
            lat_list.append(lat)
            lng_list.append(long)
        except:
            lat_list.append(0)
            lng_list.append(0)
        time.sleep(2)

100%|██████████| 340/340 [13:52<00:00,  2.45s/it]


In [20]:
df['lat'] = lat_list
df['lng'] = lng_list

In [21]:
df.to_csv('tuguegeocoded_locations_v3.csv')

## GEOPANDAS

In [29]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

#### Converting our scraped data to geopandas format

In [80]:
df = pd.read_csv('tuguegeocoded_locations_v3.csv')

In [81]:
df2 = pd.read_csv('cagayan_other_locations_v1.csv')

In [82]:
df = df[['address','AREA','NOTES',"Contact Number\n  ANOTHER NOTE: Countless rescuers na ang namatay dahil naelectrocute ng generators. INFORM PEOPLE SA CAGAYAN TO TURN OFF THEIR GENERATORS. Buhay muna ng rescuers bago ang kuryente!!", 'PERSONS TO RESCUE', 'DATE AND TIME CONTACTED','lat','lng']]

In [83]:
df2 = df2[['address','AREA','NOTES','CONTACT NUMBER','PERSONS TO RESCUE','DATE AND TIME CONTACTED', 'lat','lng']]

In [84]:
df.columns = df2.columns

In [85]:
df = df2.append(df)

In [86]:
# Converting the 
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)

In [87]:
df = df[(df['lng'] >= 120.68) & (df['lng'] <= 122.4)]
df = df[(df['lat'] >= 17.5) & (df['lat'] <= 19.6)]

In [88]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lng, df.lat))

# Mapping

In [52]:
import folium
import folium.plugins as plugins


In [89]:
df = df.reset_index(drop = True)

In [90]:
df1 = df[~df.geometry.duplicated()]
df2 = df[df.geometry.duplicated()]

In [91]:
coo = []
info = []

for geo in df2['geometry'].unique():
    coo.append(geo)
    info.append(df[df['geometry'] == geo][['address', 'AREA','NOTES','CONTACT NUMBER', 'PERSONS TO RESCUE','DATE AND TIME CONTACTED']].to_html())
    
tables = pd.DataFrame({'geometry': coo, 'info': info})

coo = []
info = []

for geo in df1['geometry'].unique():
    coo.append(geo)
    info.append(df[df['geometry'] == geo][['address', 'AREA','NOTES','CONTACT NUMBER', 'PERSONS TO RESCUE','DATE AND TIME CONTACTED']].to_html())
    
tables = tables.append(pd.DataFrame({'geometry': coo, 'info': info}))

In [92]:
tables = tables.reset_index(drop = True)
tables = gpd.GeoDataFrame(tables, geometry='geometry')


In [93]:
gdf = gdf.merge(tables, on='geometry').drop_duplicates().reset_index(drop = 'True')

In [74]:
# df['info'] = [' DETAILS: '.join(i) for i in zip(df['address'].astype(str),df['NOTES'].astype(str))]
# df['info'] = [' AREA: '.join(i) for i in zip(df['info'].astype(str),df['AREA'].astype(str))]
# df['info'] = [' CONTACT NUMBER: '.join(i) for i in zip(df['info'].astype(str),df["CONTACT NUMBER"].astype(str))]
# df['info'] = [' PERSONS TO RESCUE: '.join(i) for i in zip(df['info'].astype(str),df["PERSONS TO RESCUE"].astype(str))]
# df['info'] = [' DATE AND TIME CONTACTED: '.join(i) for i in zip(df['info'].astype(str),df["DATE AND TIME CONTACTED"].astype(str))]
# df['info'] = 'ADDRESS: ' + df['info']

In [75]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [94]:
gdf['test'] = "<div style='height:200px;overflow:auto;'>"+ gdf['info'] +"</div>" 

In [95]:
gdf.head()

,address,AREA,NOTES,CONTACT NUMBER,PERSONS TO RESCUE,DATE AND TIME CONTACTED,lat,lng,geometry,info,test
0,brgy. damurog zone 4,Alcala,NaN,NaN,NaN,NaN,17.878753,121.665383,POINT (121.66538 17.87875),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:200px;overflow:auto;'><tabl...
1,Zone2,Alcala,di makontact kanina pa tanghali,9751500123,Jeloubelle Conceja,NaN,17.904972,121.651126,POINT (121.65113 17.90497),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:200px;overflow:auto;'><tabl...
2,NaN,Alcala,NaN,NaN,Claudilyn,NaN,17.904972,121.651126,POINT (121.65113 17.90497),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:200px;overflow:auto;'><tabl...
3,Baybayog,Alcala,with 2 yr old daughter,NaN,"Castro, Sheena",NaN,17.887315,121.705295,POINT (121.70530 17.88732),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:200px;overflow:auto;'><tabl...
4,"Zone 2, Alcala","Alcala, Zone 2, Calantac",May 2 bata,9751500123,"Conceja, Jeloubelle",NaN,17.896843,121.725246,POINT (121.72525 17.89684),"<table border=""1"" class=""dataframe"">\n <thead...",<div style='height:200px;overflow:auto;'><tabl...


In [ ]:
map1 = folium.Map(
    location=[17.6245371,121.7259032 ],
    zoom_start=10,
)


loc = 'Tuguegarao Rescue Request Mapping'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

loc = 'Last Updated: 8:47 AM 11-19-2020 <br> Mapped using Google Maps Geocoding.<br> Note that these are rough estimates of the coordinates <br> as identified by the Google API based from address description.<br> Data Source: <a href = "https://docs.google.com/spreadsheets/d/1WijcCj3nlbKPESdlcdotoP4H1hLyS3RwxJIkN_kUP3E/htmlview#"> CaVolun Initiative </a> <br> Generated by <a href = "https://github.com/magoanalytics/cagayan-rescue-mapping">Mago Analytics </a>'
title_html = '''
             <p align="center" style="font-size:9">{}</p>
             '''.format(loc)   

map1.get_root().html.add_child(folium.Element(title_html))

# Add custom basemaps
basemaps['Google Maps'].add_to(map1)

gdf.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]], popup = row['test'],fill_color='red', height=100).add_to(map1), axis=1)
map1

In [189]:
map1.save('tuguegarao_v2.html')

## --------------------------------------------

## Other Districts

In [23]:
import os

In [63]:
df = pd.DataFrame()
for file in os.listdir('./Cagayan Districts Data'):
    if '.csv' in file:
        to_append = pd.read_csv('Cagayan Districts Data/' + file)[['LOCATION','AREA','REVISED ADDRESS','STATUS','RESCUE STATUS','PERSONS TO RESCUE', 'CONTACT NUMBER']]
        df = df.append(to_append)

In [76]:
lat_list = []
lng_list = []
with tqdm(total=df.shape[0]) as pbar:
    for index, row in df.iterrows():
        pbar.update(1)
                
        try:
            if pd.isnull(row['REVISED ADDRESS']):
                if pd.isnull(row['LOCATION']) == False:
                    address = row['LOCATION']
                else:
                    address = ''
            else:
                address = row['REVISED ADDRESS']
        except:
            address = row['AREA']
            
            
        if pd.isnull(row['AREA']) == False:
            address = address + ', ' + row['AREA']
            
        address = address + ', Cagayan'
            
#         print(address)
                    
        geocode_result = gmaps.geocode(address)
        try:
            lat = geocode_result[0]['geometry']['location']['lat']
            long = geocode_result[0]['geometry']['location']['lng']
            
            lat_list.append(lat)
            lng_list.append(long)
        except:
            lat_list.append(0)
            lng_list.append(0)
        time.sleep(2)

100%|██████████| 108/108 [04:21<00:00,  2.42s/it]


In [77]:
df['lat'] = lat_list
df['lng'] = lng_list

In [80]:
df.columns = (['address', 'AREA', 'REVISED ADDRESS','NOTES','RESCUE STATUS', 'PERSONS TO RESCUE', 'CONTACT NUMBER', 'lat','lng'])
df['DATE AND TIME CONTACTED'] = ''
df = df[df['RESCUE STATUS'].str.contains('Not')]

In [86]:
df.to_csv('cagayan_other_locations_v1.csv')